<a href="https://colab.research.google.com/github/jayho2026/google-colab-files/blob/main/taxi_q_learning_reinforcement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created by JJ

2023

With a lot of reference to:

https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

https://www.gymlibrary.dev/environments/toy_text/taxi/

https://github.com/openai/gym/blob/master/gym/core.py

https://medium.com/analytics-vidhya/rendering-openai-gym-environments-in-google-colab-9df4e7d6f99f



In [ ]:
!pip install cmake 'gym[atari]' scipy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install -U colabgymrender

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gym[toy_text]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 78.6 MB/s eta 0:00:00


In [ ]:
!pip3 install imageio==2.4.1



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303886 sha256=f073fe1763b4a9b3ffaae782f564e073f6c83f4300613da861bfa0df070c983d
  Stored in directory: /root/.cache/pip/wheels/be/7b/04/4d8d56f1d503e5c404f0de6018c0cfa592c71588a39b49e002
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0


In [ ]:
import gym
from colabgymrender.recorder import Recorder



env = gym.make("Taxi-v3")
renv = Recorder(env, './videos') # save a video file everytime renv.reset and renv.step is called


observation = renv.reset() # resets/restart the environment with new positions

terminal = False   # to determine whether the game have ended

# random moves
while not terminal:

  action = renv.action_space.sample()
  oberservation, reward, terminal, info = renv.step(action)

  

# play the last saved video
renv.play();

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3719168/45929032 bytes (8.1%)7962624/45929032 bytes (17.3%)10829824/45929032 bytes (23.6%)13500416/45929032 bytes (29.4%)15704064/45929032 bytes (34.2%)17588224/45929032 bytes (38.3%)19292160/45929032 bytes (42.0%)22126592/45929032 bytes (48.2%)23650304/45929032 bytes (51.5%)26255360/45929032 bytes (57.2%)27713536/45929032 bytes (60.3%)29007872/45929032 bytes (63.2%)

  deprecation(

  deprecation(

See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(

  result = np.fromstring(s, dtype='uint8')



In [1]:
# visualisation of acrions
print(renv.action_space.sample());
print(renv.step(action));

print("Action Space {}".format(renv.action_space))
print("State Space {}".format(renv.observation_space))

NameError: ignored

In [ ]:
# create q table
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])
print(q_table)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [ ]:
# Training

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1  # learning rate
gamma = 0.6  # discount rate (balances our immediate and future reward)
epsilon = 0.1 # how often it should randomly explore or use the best value available in the table

# For plotting metrics
all_epochs = []
all_penalties = []

# Start of the Q-learning algorithm
for i in range(1, 501): #501 trainings

    if i % 100 == 0:

      state = renv.reset()

      epochs, penalties, reward, = 0, 0, 0
      done = False
    
      while not done:
          if random.uniform(0, 1) < epsilon:
              action = renv.action_space.sample() # Explore action space
          else:
              action = np.argmax(q_table[state]) # Exploit learned values

          next_state, reward, done, info = renv.step(action) 
          
          old_value = q_table[state, action]
          next_max = np.max(q_table[next_state])
          
          new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
          q_table[state, action] = new_value

          if reward == -10:
              penalties += 1

          state = next_state
          epochs += 1
          
          clear_output(wait=True)

          print(f"Episode: {i}, Saved to videos")
    
    else:
      state = env.reset()

      epochs, penalties, reward, = 0, 0, 0
      done = False
    
      while not done:
          if random.uniform(0, 1) < epsilon:
              action = env.action_space.sample() # Explore action space
          else:
              action = np.argmax(q_table[state]) # Exploit learned values

          next_state, reward, done, info = env.step(action) 
          
          old_value = q_table[state, action]
          next_max = np.max(q_table[next_state])
          
          new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
          q_table[state, action] = new_value

          if reward == -10:
              penalties += 1

          state = next_state
          epochs += 1
        
    

print("Training finished.\n")

Episode: 500, Saved to videos
Training finished.



In [ ]:
print(q_table)

[[  0.           0.           0.           0.           0.
    0.        ]
 [ -2.418369    -2.36395077  -2.4183705   -2.36395098  -2.27325184
  -11.36384242]
 [ -1.87014265  -1.45024336  -1.87014362  -1.45024207  -0.7504
  -10.45022946]
 ...
 [ -1.044785     0.416       -0.98172076  -1.24332489  -6.12005051
   -6.03597564]
 [ -2.14605738  -2.1218174   -2.16939092  -2.12181446  -5.61362008
   -6.24587278]
 [  1.7895782    1.3051951    2.08119955  11.          -1.98311649
   -2.93786053]]


In [ ]:
# Zip video folder to be downloaded
!zip -r ./video_data.zip ./videos

In [ ]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 196.16
Average penalties per episode: 0.0
